### Задача 4.2
Рассмотрим некоторую задачу классификации. Пусть задано качество 4 моделей $a_1, a_2, a_3, a_4$. Качество полученных моделей показано в таблице.

Исследователю требуется выбрать наилучшую модель. Для выбора лучшей модели исследовать требуется попарно сравнить среднее значение качества всех моделей. Может ли исследователь утверждать что какая-то из моделей лучше другой?

Требуется:

* записать задачу формально;
* предложить статистику для решения данной задачи;
* записать нулевое распределение данной статистики;
* записать явно правило принятия решения на основе статистики и нулевого распределения для обеспечения уровня значимости $\alpha = 0.05$;
* проверить гипотезу по записанному критерию, для данных из условия. Противоречат ли они гипотезе?

Все выкладки должны быть сделаны аналитически, без использования компьютера. (допускается использование компютера для подстановвки численых значений в финальную формулу)

In [1]:
import pandas as pd
import scipy.stats as st
from itertools import combinations
from  statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.multitest import multipletests

classifiers = pd.read_csv('data/classifiers.csv', sep = ',', index_col = 'Номер выборки')
classifiers

,a1,a2,a3,a4
Номер выборки,,,,
1,86,50,93,13
2,85,74,55,35
3,53,92,58,51
4,44,41,56,37
5,2,18,99,26
6,5,68,35,17


In [2]:
classifiers.describe().loc['mean']

a1    45.833333
a2    57.166667
a3    66.000000
a4    29.833333
Name: mean, dtype: float64

In [3]:
classifiers_np = classifiers.to_numpy().T

In [4]:
for i, x in enumerate(classifiers_np):
    print('p_value for classifier {} (Shapiro, Jarque-Bera) = ({}, {})'.format(i+1, st.shapiro(x)[-1], st.jarque_bera(x)[-1]))

y = classifiers_np.flatten()
print('p_value for all classifiers (Shapiro, Jarque-Bera) = ({}, {})'.format(st.shapiro(y)[-1], st.jarque_bera(y)[-1]))

p_value for classifier 1 (Shapiro, Jarque-Bera) = (0.26456114649772644, 0.7459629534933241)
p_value for classifier 2 (Shapiro, Jarque-Bera) = (0.9791138768196106, 0.865335484560948)
p_value for classifier 3 (Shapiro, Jarque-Bera) = (0.2797580063343048, 0.7677030867299224)
p_value for classifier 4 (Shapiro, Jarque-Bera) = (0.8345257043838501, 0.8381024992031822)
p_value for all classifiers (Shapiro, Jarque-Bera) = (0.4981343448162079, 0.6286467415823447)


Исходя из этого можно считать, что результаты каждого классификатора подчиняются нормальному распределению.

Следовательно, средние значения классификаторов можно сравнивать  попарно с помощью Т-теста. Также можно использовать критерий знаковых раногов Уилкоксона (для этого распределения не обязательно должны быть нормальными) 

Формальная запись задачи: дана выборка $X_1, X_2, X_3, X_4$, где $X_i$ - качество классификации $i$-ой модели. Размер $X_i$ равен $6 \; \forall i$ (6 измерений для кажой модели)

По условию не совсем понятно, чего мы именно хотим. Выбрать лучшую модель из всех? (ели верить фразе "Для выбора лучшей модели исследователю требуется попарно сравнить среднее значение качества всех моделей"). Или выбрать модель, которая будет лучше какой-то другой? (если верить фразе "Может ли исследователь утверждать что какая-то из моделей лучше другой?")

Если мы хотим проверить утверждение о том,что существуют 2 модели, качество одной из которых выше, чем у другой, то:

$H_0: \; \exists i, j \; (i \ne j): \; E(X_i)>E(X_j)$

$H_1:$ нулевая гипотеза неверна 

Если же мы хотим проверить утверждение о том, что существует модель, которая лучше, чем всё другие, то:

$H_0: \; \exists i: \; E(X_i)>E(X_j) \; \forall j: \; i \ne j$

$H_1:$ нулевая гипотеза неверна

Логичнее было бы проверять именно вторую второй вариант - так и сделаем. 

Будем проверять гипотезу попарным сравнением среднего качества моделей моделей с помощью T-теста (с правосторонней альтернативой). Так как данная задача является $\textbf{задачей множественного тестирования гипотез}$, то нам нужно будет учесть поправку на $p\_value$

Статистика T-test для двух выборок: $$T(X_1^{n_1}, X_2^{n_1}) = \frac{\overline{X_1}-\overline{X_2}}{\sqrt{\frac{S_1^2}{n_1} + \frac{S_2^2}{n_2}}}$$ ????

In [5]:
def test(data, testtype='ttest'):
    rejections, p_values = 0, []
    for x, y in combinations(data, 2):
        if testtype=='wilcoxon':
            p_value = st.wilcoxon(x, y, alternative='greater')[1]
        elif testtype=='ttest':
            p_value = st.ttest_ind(x, y, alternative='greater')[1]
        else:
            raise ValueError("There is no such test!")
        p_values.append(p_value)
        rejections += p_value <= 0.05
    return rejections, p_values

Так как в данном случае не отвергается гипотеза о том, что данные распределены нормально, любая из поправок (Бонферрнони, Холма, Шидака, Шидака-Холма, Бенджамини-Хохберга (про Бенджамини-Иекутиели я достоверно не знаю)) будет обеспечивать контроль над FWER на уровне 0.05 (выбранный уровень значимости)

In [6]:
methods = ['bonferroni', 'holm', 'sidak', 'holm-sidak', 'fdr_bh'] # 'fdr_by'
p_values = test(classifiers_np, 'wilcoxon')[1]

for method in methods:
    reject_array, p_vals_corr, *_ = multipletests(p_values, method = method)
    print('Rejections with {} = {} ---reject_array = {}\n'.format(method, sum(reject_array), reject_array))

Rejections with bonferroni = 0 ---reject_array = [False False False False False False]

Rejections with holm = 0 ---reject_array = [False False False False False False]

Rejections with sidak = 0 ---reject_array = [False False False False False False]

Rejections with holm-sidak = 0 ---reject_array = [False False False False False False]

Rejections with fdr_bh = 0 ---reject_array = [False False False False False False]



При использовании критерия знаковых раногов Уилкоксона для попарного сравнения средних значений качества моделей отклоняется гипотеза о том, что есть модели с лучшим средним качеством (причём для любого из методов поправок)

In [7]:
methods = ['bonferroni', 'holm', 'sidak', 'holm-sidak', 'fdr_bh'] # 'fdr_by'
p_values = test(classifiers_np, 'ttest')[1]

for method in methods:
    reject_array, p_vals_corr, *_ = multipletests(p_values, method = method)
    print('Rejections with {} = {} ---reject_array = {}\n'.format(method, sum(reject_array), reject_array))

Rejections with bonferroni = 1 ---reject_array = [False False False False False  True]

Rejections with holm = 1 ---reject_array = [False False False False False  True]

Rejections with sidak = 1 ---reject_array = [False False False False False  True]

Rejections with holm-sidak = 1 ---reject_array = [False False False False False  True]

Rejections with fdr_bh = 1 ---reject_array = [False False False False False  True]



При использовании T-теста для попарного сравнения средних значений качества моделей не отвергается гипотеза о том, что есть 2 модели, качество одной из которых выше, чем у другой (причём для любого из методов поправок). Зная как именно работет itertools.combinations, можно сказать, что не отвергается гипотеза о том, что среднее качество третьей модели выше, чем у четвёртой. 

То есть мы можем утвержать, что третья модель лучше четвёртой. 

Попробуем отдельно сравнить третью модель с остальными - у неё самое большое среднее значение качества (это уже не относится напрямую к решению задачи)

In [8]:
def test_1vsall(data):
    rejections, p_values = 0, []
    for i, x in enumerate(data):
        if i!=2:
            p_value = st.wilcoxon(data[2], x, alternative='greater')[1]
#             p_value = st.ttest_ind(data[2], x, alternative='greater')[1]
            p_values.append(p_value)
            rejections += p_value <= 0.05
    return rejections, p_values

methods = ['bonferroni', 'holm', 'sidak', 'holm-sidak', 'fdr_bh'] # 'fdr_by'
p_values = test_1vsall(classifiers_np)[1]

for method in methods:
    reject_array, p_vals_corr, *_ = multipletests(p_values, method = method)
    print('Rejections with {} = {}, reject_array = {}\n'.format(method, sum(reject_array), reject_array))

Rejections with bonferroni = 1, reject_array = [False False  True]

Rejections with holm = 1, reject_array = [False False  True]

Rejections with sidak = 1, reject_array = [False False  True]

Rejections with holm-sidak = 1, reject_array = [False False  True]

Rejections with fdr_bh = 1, reject_array = [False False  True]



В этом случае получается, третья модель лучше четвертой, но не лучше других. Так что получили аналогичный результат. 